In [1]:
from read_data import read_anomaly_detection_dataset
import pandas as pd

In [2]:
df = read_anomaly_detection_dataset()

In [3]:
df_renta=pd.read_csv("input/datos_renta.csv", encoding='latin-1', sep=';')

In [4]:
df.columns

Index(['ID', 'Caracteristicas', 'Habitaciones', 'Aseos', 'Terraza', 'Piscina',
       'Garaje', 'Precio', 'Metros', 'CodigoPostal', 'Latitud', 'Longitud',
       'NPRO', 'NCA', 'NMUN', 'PrecioM2', 'CUDIS'],
      dtype='object')

In [5]:
def clean_renta(df_renta):
    df_renta = df_renta[["Distritos", "Indicadores de renta media", "Periodo", "Total"]]

    #Limpiar distritos
    df_renta = df_renta.dropna(subset=['Distritos'])
    df_renta["Distritos"] = df_renta["Distritos"].str.split(" ").str[0]
    df_renta = df_renta[df_renta["Distritos"].apply(pd.to_numeric, errors='coerce').notna()].dropna()

    #Filtrar datos
    df_renta = df_renta[df_renta["Periodo"]==2020]
    df_renta.drop("Periodo", axis=1, inplace=True)
    df_renta = df_renta[(df_renta["Indicadores de renta media"]=="Renta bruta media por persona") | (df_renta["Indicadores de renta media"]=="Renta bruta media por hogar")]
    
    df_renta = df_renta.dropna(subset=['Total'])
    df_renta["Total"]=df_renta["Total"].str.replace('.','')
    df_renta = df_renta[df_renta["Total"].apply(pd.to_numeric, errors='coerce').notna()].dropna()
    df_renta["Total"] = df_renta["Total"].astype(int)

    df_renta= df_renta.pivot_table(index=['Distritos'], columns='Indicadores de renta media', values='Total',aggfunc='mean').reset_index()
    return df_renta

In [6]:
df_renta_c = clean_renta(df_renta)

In [7]:
# Agregar renta al df
df_merged = pd.merge(df, df_renta_c, how='left', left_on='CUDIS', right_on='Distritos').drop("Distritos", axis=1)
df_merged

,ID,Caracteristicas,Habitaciones,Aseos,Terraza,Piscina,Garaje,Precio,Metros,CodigoPostal,Latitud,Longitud,NPRO,NCA,NMUN,PrecioM2,CUDIS,Renta bruta media por hogar,Renta bruta media por persona
0,116893306,Apartment,1,1,0,0,1,139000,40,25530,42.70474,0.793855,Lleida,Cataluña,Vielha e Mijaran,3475.000000,2524301,40182.000000,16694.000000
1,118596151,Apartment,3,1,0,0,0,74000,68,43205,41.15317,1.108049,Tarragona,Cataluña,Reus,1088.235294,4312303,32367.250000,12348.250000
2,118720927,House,4,2,1,0,0,228000,178,43540,40.61473,0.588369,Tarragona,Cataluña,Sant Carles de la Ràpita,1280.898876,4313602,34605.666667,14264.333333
3,120037507,Apartment,2,1,1,1,1,192300,75,43300,41.02861,0.936620,Tarragona,Cataluña,Mont-roig del Camp,2564.000000,4309201,32511.666667,13705.222222
4,121155231,House,4,2,1,0,0,143200,134,43412,41.45768,1.177539,Tarragona,Cataluña,Solivella,1068.656716,4314701,41662.000000,16834.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68223,162446507,Apartment,2,2,1,0,0,175000,89,08242,41.73527,1.825772,Barcelona,Cataluña,Manresa,1966.292135,0811302,44261.466667,17845.133333
68224,162446510,Apartment,3,2,1,0,0,106000,70,08670,41.89949,1.873544,Barcelona,Cataluña,Navàs,1514.285714,0814101,40101.200000,16255.800000
68225,162446526,Apartment,4,1,0,0,0,125000,86,43001,41.11553,1.247802,Tarragona,Cataluña,Tarragona,1453.488372,4314806,40861.866667,16366.600000
68226,162446537,Apartment,4,3,0,0,0,477000,170,08201,41.55113,2.104109,Barcelona,Cataluña,Sabadell,2805.882353,0818701,55412.461538,22014.615385


In [8]:
df_merged=df_merged.rename({"Renta bruta media por hogar":"RentaBrutaHogar","Renta bruta media por persona":"RentaBrutaPersona"}, axis=1)

In [9]:
df_merged

,ID,Caracteristicas,Habitaciones,Aseos,Terraza,Piscina,Garaje,Precio,Metros,CodigoPostal,Latitud,Longitud,NPRO,NCA,NMUN,PrecioM2,CUDIS,RentaBrutaHogar,RentaBrutaPersona
0,116893306,Apartment,1,1,0,0,1,139000,40,25530,42.70474,0.793855,Lleida,Cataluña,Vielha e Mijaran,3475.000000,2524301,40182.000000,16694.000000
1,118596151,Apartment,3,1,0,0,0,74000,68,43205,41.15317,1.108049,Tarragona,Cataluña,Reus,1088.235294,4312303,32367.250000,12348.250000
2,118720927,House,4,2,1,0,0,228000,178,43540,40.61473,0.588369,Tarragona,Cataluña,Sant Carles de la Ràpita,1280.898876,4313602,34605.666667,14264.333333
3,120037507,Apartment,2,1,1,1,1,192300,75,43300,41.02861,0.936620,Tarragona,Cataluña,Mont-roig del Camp,2564.000000,4309201,32511.666667,13705.222222
4,121155231,House,4,2,1,0,0,143200,134,43412,41.45768,1.177539,Tarragona,Cataluña,Solivella,1068.656716,4314701,41662.000000,16834.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68223,162446507,Apartment,2,2,1,0,0,175000,89,08242,41.73527,1.825772,Barcelona,Cataluña,Manresa,1966.292135,0811302,44261.466667,17845.133333
68224,162446510,Apartment,3,2,1,0,0,106000,70,08670,41.89949,1.873544,Barcelona,Cataluña,Navàs,1514.285714,0814101,40101.200000,16255.800000
68225,162446526,Apartment,4,1,0,0,0,125000,86,43001,41.11553,1.247802,Tarragona,Cataluña,Tarragona,1453.488372,4314806,40861.866667,16366.600000
68226,162446537,Apartment,4,3,0,0,0,477000,170,08201,41.55113,2.104109,Barcelona,Cataluña,Sabadell,2805.882353,0818701,55412.461538,22014.615385


In [10]:
# Fillna con el "median"
df_merged['RentaBrutaHogar'].fillna(df_merged['RentaBrutaHogar'].median(), inplace=True)
df_merged['RentaBrutaPersona'].fillna(df_merged['RentaBrutaPersona'].median(), inplace=True)

C:\Users\emebe\AppData\Local\Temp\ipykernel_1652\909651797.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_merged['RentaBrutaHogar'].fillna(df_merged['RentaBrutaHogar'].median(), inplace=True)
C:\Users\emebe\AppData\Local\Temp\ipykernel_1652\909651797.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting value

In [11]:
df_density = pd.read_csv("input/densidad_trabajo.csv", on_bad_lines='skip', sep=';')
df_density

,Código,Etiqueta,Población 2023,% Trabajadores en servicios 2023,Densidad de viviendas familiares 2021,% Trabajadores en agricultura 2023,% Trabajadores en industria 2023,% Trabajadores en construcción 2023
0,1001,Alegría-Dulantzi,2969.0,60.9,64.7,2.3,28.6,8.2
1,1002,Amurrio,10299.0,38.3,51.4,2.1,53.6,6.0
2,1003,Aramaio,1409.0,55.7,9.9,9.7,33.8,0.8
3,1004,Artziniega,1832.0,43.7,41.3,3.0,44.4,8.9
4,1006,Armiñón,233.0,10.9,15.3,2.7,82.3,4.1
...,...,...,...,...,...,...,...,...
8212,54001,Isla del Perejil,NaN,NaN,NaN,NaN,NaN,NaN
8213,54002,Peñon de Vélez de la Gomera,NaN,NaN,NaN,NaN,NaN,NaN
8214,54003,Islas Chafarinas,NaN,NaN,NaN,NaN,NaN,NaN
8215,54004,Islas Alhucemas,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
df_density = df_density.rename({"Código":"CUDIS", 
                   "Población 2023": "poblacion_2023",
                   "Densidad de viviendas familiares 2021" : "densidad_inm_m2",
                  "% Trabajadores en servicios 2023" : "%_servicios",
                  "% Trabajadores en industria 2023" : "%_industria",
                  "% Trabajadores en agricultura 2023" : "%_agricultura",
                  "% Trabajadores en construcción 2023" : "%_construccion",}, axis=1)

In [14]:
df_density = df_density.dropna(subset=['CUDIS'])
df_density["CUDIS"] = df_density["CUDIS"].astype(int).astype(str)
df_density['CUDIS'] = df_density['CUDIS'].str.zfill(5)

In [15]:
df_density['CUDIS']

0       01001
1       01002
2       01003
3       01004
4       01006
        ...  
8212    54001
8213    54002
8214    54003
8215    54004
8216    54005
Name: CUDIS, Length: 8217, dtype: object

In [16]:
df_merged["MUN_CUDIS"] = df_merged["CUDIS"].str[:-2]
df_merged["MUN_CUDIS"]

0        25243
1        43123
2        43136
3        43092
4        43147
         ...  
68223    08113
68224    08141
68225    43148
68226    08187
68227    08089
Name: MUN_CUDIS, Length: 68228, dtype: object

In [17]:
df_merged = pd.merge(df_merged, df_density, how='left', left_on='MUN_CUDIS', right_on='CUDIS').drop("MUN_CUDIS", axis=1).drop("CUDIS_x", axis=1)
df_merged

,ID,Caracteristicas,Habitaciones,Aseos,Terraza,Piscina,Garaje,Precio,Metros,CodigoPostal,...,RentaBrutaHogar,RentaBrutaPersona,CUDIS_y,Etiqueta,poblacion_2023,%_servicios,densidad_inm_m2,%_agricultura,%_industria,%_construccion
0,116893306,Apartment,1,1,0,0,1,139000,40,25530,...,40182.000000,16694.000000,25243,Vielha e Mijaran,5740.0,83.5,25.1,0.4,3.2,13.0
1,118596151,Apartment,3,1,0,0,0,74000,68,43205,...,32367.250000,12348.250000,43123,Reus,108535.0,80.1,968.1,1.6,11.1,7.2
2,118720927,House,4,2,1,0,0,228000,178,43540,...,34605.666667,14264.333333,43136,Sant Carles de la Ràpita,15329.0,75.4,226.8,8.1,9.8,6.7
3,120037507,Apartment,2,1,1,1,1,192300,75,43300,...,32511.666667,13705.222222,43092,Mont-roig del Camp,13682.0,66.2,262.1,9.6,9.0,15.2
4,121155231,House,4,2,1,0,0,143200,134,43412,...,41662.000000,16834.000000,43147,Solivella,632.0,45.4,23.1,36.1,4.6,13.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68223,162446507,Apartment,2,2,1,0,0,175000,89,08242,...,44261.466667,17845.133333,08113,Manresa,78570.0,80.7,963.9,0.5,13.4,5.4
68224,162446510,Apartment,3,2,1,0,0,106000,70,08670,...,40101.200000,16255.800000,08141,Navàs,6161.0,61.5,38.3,4.4,19.6,14.5
68225,162446526,Apartment,4,1,0,0,0,125000,86,43001,...,40861.866667,16366.600000,43148,Tarragona,138326.0,87.2,1203.1,0.4,7.6,4.8
68226,162446537,Apartment,4,3,0,0,0,477000,170,08201,...,55412.461538,22014.615385,08187,Sabadell,217968.0,83.4,2471.9,0.1,9.8,6.6


In [18]:
df_merged = df_merged.rename({"CUDIS_y" : "CUDIS"}, axis=1)

In [19]:
df_merged.drop("Etiqueta", axis=1, inplace=True)

In [20]:
df_merged.columns

Index(['ID', 'Caracteristicas', 'Habitaciones', 'Aseos', 'Terraza', 'Piscina',
       'Garaje', 'Precio', 'Metros', 'CodigoPostal', 'Latitud', 'Longitud',
       'NPRO', 'NCA', 'NMUN', 'PrecioM2', 'RentaBrutaHogar',
       'RentaBrutaPersona', 'CUDIS', 'poblacion_2023', '%_servicios',
       'densidad_inm_m2', '%_agricultura', '%_industria', '%_construccion'],
      dtype='object')

In [21]:
df_merged.to_csv("output/enhanced.csv",index=False)